# 🔍 Image Similarity with Pretrained CNN and Fine-Tuning
Questo notebook esegue fine-tuning su un dataset etichettato di immagini, quindi usa il modello per recuperare le k immagini più simili dalla gallery per ciascuna immagine di query.

In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from sklearn.neighbors import NearestNeighbors
import numpy as np
import json
import os
from PIL import Image
from tqdm import tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [4]:
def get_model(num_classes):
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


In [5]:
def train_model(model, dataloader, epochs=5, lr=1e-4):
    model = model.to(device)
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Loss: {running_loss/len(dataloader):.4f}")
    return model


In [6]:
def get_feature_extractor(trained_model):
    feature_extractor = nn.Sequential(*list(trained_model.children())[:-1])
    feature_extractor.eval()
    return feature_extractor.to(device)


In [7]:
def extract_embeddings_from_folder(folder_path, model):
    image_paths = sorted([os.path.join(folder_path, fname)
                          for fname in os.listdir(folder_path)
                          if fname.lower().endswith(('.jpg', '.jpeg', '.png'))])

    all_embeddings = []
    filenames = []

    with torch.no_grad():
        for i in range(0, len(image_paths), 32):
            batch_paths = image_paths[i:i+32]
            imgs = [transform(Image.open(p).convert("RGB")) for p in batch_paths]
            imgs = torch.stack(imgs).to(device)
            vecs = model(imgs).squeeze(-1).squeeze(-1)
            all_embeddings.append(vecs.cpu())
            filenames.extend(batch_paths)

    return torch.cat(all_embeddings, dim=0).numpy(), filenames


In [8]:
def retrieve_query_vs_gallery(query_embs, query_files, gallery_embs, gallery_files, k=5):
    model = NearestNeighbors(n_neighbors=k, metric='cosine')
    model.fit(gallery_embs)
    distances, indices = model.kneighbors(query_embs)

    results = []
    for i, query_path in enumerate(query_files):
        query_rel = query_path.replace("\\", "/")
        gallery_matches = [gallery_files[idx].replace("\\", "/") for idx in indices[i]]
        results.append({
            "filename": query_rel,
            "gallery_images": gallery_matches
        })
    return results


In [11]:
def save_submission(results, output_path):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)

## 🧪 Esecuzione completa

In [13]:
# Step 1: Fine-tune il modello sul training set
train_dataset = datasets.ImageFolder("C:/Users/utente/Desktop/UNITN/Intro to ML/ML-project/Examples/training", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
model = get_model(num_classes=len(train_dataset.classes))
model = train_model(model, train_loader, epochs=5)

# Step 2: Estrai features da query e gallery
feature_extractor = get_feature_extractor(model)
query_embeddings, query_files = extract_embeddings_from_folder("C:/Users/utente/Desktop/UNITN/Intro to ML/ML-project/Examples/test/query", feature_extractor)
gallery_embeddings, gallery_files = extract_embeddings_from_folder("C:/Users/utente/Desktop/UNITN/Intro to ML/ML-project/Examples/test/gallery", feature_extractor)

# Step 3: Retrieval
submission = retrieve_query_vs_gallery(query_embeddings, query_files, gallery_embeddings, gallery_files, k=3)

# Step 4: Salvataggio nella repo
submission_path = "C:/Users/utente/Desktop/UNITN/Intro to ML/ML-project/submission/submission.json"

save_submission(submission, submission_path)
print(f"✅ Submission salvata in: {submission_path}")


c:\Users\utente\anaconda3\envs\ElisaML\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\utente\anaconda3\envs\ElisaML\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/5: 100%|██████████| 2/2 [00:07<00:00,  3.92s/it]


Loss: 1.0978


Epoch 2/5: 100%|██████████| 2/2 [00:07<00:00,  3.65s/it]


Loss: 0.5315


Epoch 3/5: 100%|██████████| 2/2 [00:07<00:00,  3.54s/it]


Loss: 0.1984


Epoch 4/5: 100%|██████████| 2/2 [00:07<00:00,  3.78s/it]


Loss: 0.1186


Epoch 5/5: 100%|██████████| 2/2 [00:07<00:00,  3.68s/it]


Loss: 0.0739
✅ Submission salvata in: C:/Users/utente/Desktop/UNITN/Intro to ML/ML-project/submission/submission.json
